In [1]:
import torch
import torch.multiprocessing as mp

In [2]:
from rsnn.spike_sequences.sampling import sample_spike_sequences
from rsnn.spike_sequences.template import segment_spike_sequence
from rsnn.optimization.utils import compute_observation_matrices
from rsnn.optimization.optimization import compute_weights

In [3]:
L = 100
T = 100
Tr = 20

K = 500

wb = 0.1
taub = 60
a = 1
b = 0
theta = 1.0

Tr = 20
beta = 2  # spreading hyperparameter

impulse_resp = lambda t_: (t_ >= 0) * t_ / beta * torch.exp(1 - t_ / beta)
impulse_resp_deriv = lambda t_: (t_ >= 0) * 1 / beta * (1 - t_ / beta) * torch.exp(1 - t_ / beta)

delays = torch.FloatTensor(L, K).uniform_(0, taub)
sources = torch.randint(0, L, (L, K))

spike_sequences = sample_spike_sequences(L, T, Tr)

In [4]:
device = "cpu" # "mps" if torch.has_mps else "cpu"

observation_matrices = []
for l in range(L):
    segmentation = segment_spike_sequence(spike_sequences[l], Tr, 1)
    observation_matrices.append(
        compute_observation_matrices(spike_sequences, segmentation, delays[l], sources[l], Tr, impulse_resp, impulse_resp_deriv, device)
    )

In [5]:
#weights = torch.empty(L, K, device=device)
#for l in range(L):
#    print(l)
#    weights[l] = compute_weights(*observation_matrices[l], wb, theta, a, b)
    
# serial took 18.1s for 10 neurons -> 180s for 100 neurons

In [6]:
manager = mp.Manager()
return_dict = manager.dict()
jobs = []
for l in range(L):
    p = mp.Process(target=compute_weights, args=(l, return_dict, *observation_matrices[l], wb, theta, a, b))
    jobs.append(p)
    p.start()

for p in jobs:
    p.join()
    
weights = torch.stack([return_dict[l] for l in range(L)])

Process Process-55:
Traceback (most recent call last):
  File "/Users/haguettaz/opt/anaconda3/envs/rsnn/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/haguettaz/opt/anaconda3/envs/rsnn/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/haguettaz/Documents/PhD/RSNN/rsnn/optimization/optimization.py", line 53, in compute_weights
    mw, Vw = compute_weight_posterior(mw_f, Vw_f, mz_b_f, Vz_b_f, C_f, mz_b_a, Vz_b_a, C_a, mz_b_s, Vz_b_s, C_s)
  File "/Users/haguettaz/Documents/PhD/RSNN/rsnn/optimization/posterior.py", line 41, in compute_weight_posterior
    Vw_f = prev_Vw_f - H @ C_f[n] @ prev_Vw_f
KeyboardInterrupt
Process Process-66:
Traceback (most recent call last):
  File "/Users/haguettaz/opt/anaconda3/envs/rsnn/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/haguettaz/opt/anaconda3/envs/rsnn/lib/python3.8/multiproces

    self.run()
  File "/Users/haguettaz/opt/anaconda3/envs/rsnn/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/haguettaz/Documents/PhD/RSNN/rsnn/optimization/optimization.py", line 53, in compute_weights
    mw, Vw = compute_weight_posterior(mw_f, Vw_f, mz_b_f, Vz_b_f, C_f, mz_b_a, Vz_b_a, C_a, mz_b_s, Vz_b_s, C_s)
  File "/Users/haguettaz/Documents/PhD/RSNN/rsnn/optimization/posterior.py", line 65, in compute_weight_posterior
    Vw_f = prev_Vw_f - H @ C_s[n] @ prev_Vw_f
KeyboardInterrupt
Process Process-21:
Traceback (most recent call last):
  File "/Users/haguettaz/opt/anaconda3/envs/rsnn/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/haguettaz/opt/anaconda3/envs/rsnn/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/haguettaz/Documents/PhD/RSNN/rsnn/optimization/optimization.py", line 53, i

KeyboardInterrupt: 

In [ ]:
# https://pytorch.org/tutorials/intermediate/dist_tuto.html